In [27]:
!pip install -U scikit-learn

In [28]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [30]:
data = pd.read_csv('trg.csv')

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
data['abstract'] = data['abstract'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

X_train, X_test, y_train, y_test = train_test_split(data['abstract'], data['class'], test_size=0.2, random_state=42)


AttributeError: module 'nltk' has no attribute 'internals'

In [31]:
# Vectorize your data
vectorizer = CountVectorizer(binary=True)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Encode your labels
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# Calculate priors
priors = np.bincount(y_train_enc) / len(y_train_enc)

# Calculate likelihoods
likelihoods = np.zeros((len(set(y_train_enc)), X_train_vec.shape[1]))
for c in range(len(set(y_train_enc))):
    likelihoods[c, :] = (X_train_vec[y_train_enc==c, :].sum(axis=0) + 1) / (np.sum(y_train_enc==c) + 1)

# Predict the class of the test set
y_pred = []
for i in range(X_test_vec.shape[0]):
    posterior = np.log(priors) + X_test_vec[i, :] @ np.log(likelihoods).T
    y_pred.append(np.argmax(posterior))

# Print the accuracy of your model
print(f'Accuracy: {accuracy_score(y_test_enc, y_pred)}')


NameError: name 'X_train' is not defined